In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dataset_dir = 'C:/Users/vsavelev/GITHUB/DS_projet/jan24_cds_mushrooms/data'

# Create ImageDataGenerator with validation split
datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

print(f'Training samples: {train_generator.samples}')
print(f'Validation samples: {validation_generator.samples}')

Found 2199 images belonging to 2 classes.
Found 549 images belonging to 2 classes.
Training samples: 2199
Validation samples: 549


In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Load and Configure the Pre-trained ResNet50 Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
"""
weights='imagenet': Loads the pre-trained weights from the ImageNet dataset.
include_top=False: Excludes the top fully-connected layers of the ResNet50 model, enabling you to add your own custom layers.
input_shape=(224, 224, 3): Specifies the input shape of the images (224x224 pixels, with 3 color channels - RGB).
"""

# Freeze the base model (to freeze the pre-trained layers)
base_model.trainable = False

# Add custom layers on top of the base model
model = models.Sequential([   #allows to stack layers linearly
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # L2 regularization
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

"""
GlobalAveragePooling2D(): Reduces each feature map to a single number by taking the average, 
which helps to reduce the size of the model and prevent overfitting.
Dense(1024, activation='relu'): Adds a fully connected layer with 1024 units and ReLU activation function.
Dropout(0.5): Adds a dropout layer with a 50% dropout rate to prevent overfitting by randomly setting half of the input units 
to 0 at each update during training.
Dense(train_generator.num_classes, activation='softmax'): 
Adds the final output layer with units equal to the number of classes in your dataset, using the softmax activation function for multi-class classification.
"""
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

"""
optimizer=tf.keras.optimizers.Adam(): Uses the Adam optimizer, which is an adaptive learning rate optimization algorithm.
loss='categorical_crossentropy': Uses categorical cross-entropy as the loss function, suitable for multi-class classification.
metrics=['accuracy']: Tracks accuracy as the metric to evaluate the model's performance during training and testing.
"""
model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [3]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

#This specifies the number of complete passes through the training dataset. Here, the model will train for 10 epochs.

Epoch 1/10


c:\Users\vsavelev\AppData\Local\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


68/68 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.8540 - loss: 6.8043 - val_accuracy: 0.8860 - val_loss: 0.6636
Epoch 2/10
 1/68 ━━━━━━━━━━━━━━━━━━━━ 2:09 2s/step - accuracy: 0.8438 - loss: 0.8550

c:\Users\vsavelev\AppData\Local\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8438 - loss: 0.8550 - val_accuracy: 1.0000 - val_loss: 0.4526
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 146s 2s/step - accuracy: 0.8873 - loss: 0.6507 - val_accuracy: 0.8879 - val_loss: 0.4938
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8750 - loss: 0.5112 - val_accuracy: 0.8000 - val_loss: 0.6924
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.8857 - loss: 0.5040 - val_accuracy: 0.8879 - val_loss: 0.4471
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8750 - loss: 0.4322 - val_accuracy: 0.8000 - val_loss: 0.6413
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.8926 - loss: 0.4378 - val_accuracy: 0.8860 - val_loss: 0.4396
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9375 - loss: 0.3477 - val_accuracy: 1.0000 - val_loss: 0.2697
Epoch 9/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 173s 3s/step - accuracy: 0.8890 - loss: 0.4132 - val_accuracy: 0.8879 - val_loss: 0.444

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# Unfreeze some layers
base_model.trainable = True
fine_tune_at = 100  # fine-tune from this layer onwards

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

#  # Unfreeze more layers gradually
# for layer in base_model.layers[:-10]:  # Unfreeze all layers except the last 10 layers
#     layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6), #learning_rate=1e-5
              loss='categorical_crossentropy', #The categorical cross-entropy loss function is used because this is a multi-class classification problem
              metrics=['accuracy'])

print("Model compiled successfully.")

"""
The Adam optimizer is used with a very small learning rate (1e-5). Fine-tuning typically 
uses a smaller learning rate to prevent large updates to the weights, which could potentially destroy the learned features in the pre-trained model.
"""

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

print("Callbacks created successfully.")


history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,
    #callbacks=[early_stopping, model_checkpoint]
)


Model compiled successfully.
Callbacks created successfully.
Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 267s 4s/step - accuracy: 0.8686 - loss: 0.6860 - val_accuracy: 0.8897 - val_loss: 0.5719
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8750 - loss: 0.6368 - val_accuracy: 0.6000 - val_loss: 0.7365
Epoch 3/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 286s 4s/step - accuracy: 0.8856 - loss: 0.6046 - val_accuracy: 0.8897 - val_loss: 0.6145
Epoch 4/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8438 - loss: 0.5484 - val_accuracy: 0.6000 - val_loss: 0.7114
Epoch 5/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 323s 5s/step - accuracy: 0.8840 - loss: 0.5082 - val_accuracy: 0.8860 - val_loss: 0.5360
Epoch 6/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8750 - loss: 0.4655 - val_accuracy: 1.0000 - val_loss: 0.4314
Epoch 7/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 299s 4s/step - accuracy: 0.8770 - loss: 0.4487 - val_accuracy: 0.8879 - val_loss: 0.4546
Epoch 8/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - a

In [13]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')


18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.8704 - loss: 0.4312
Validation loss: 0.4050602614879608
Validation accuracy: 0.8870673775672913
